In [45]:
import re, string
import random
import json as json
from nltk import word_tokenize
from nltk.corpus import twitter_samples
from nltk.tag import pos_tag
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk import FreqDist
from nltk import classify
from nltk import NaiveBayesClassifier

stop_words = stopwords.words('english')

with open('master.json') as f:
    palantir = json.load(f)
    
def tweets_to_string(data):    
    list = []   
    for key in data:
        list += [data[key]['text']]
    return list
        
palantir_tweets = tweets_to_string(palantir)

positive_tweets = twitter_samples.strings('positive_tweets.json')

negative_tweets = twitter_samples.strings('negative_tweets.json')
text = twitter_samples.strings('tweets.20150430-223406.json')

positive_tweet_tokens = twitter_samples.tokenized('positive_tweets.json')
negative_tweet_tokens = twitter_samples.tokenized('negative_tweets.json')
palantir_tweet_tokens = [word_tokenize(i) for i in palantir_tweets]

    

def lemmatize_sentence(tokens):
    lemmatizer = WordNetLemmatizer()
    lemmatized_sentence = []
    for word, tag in pos_tag(tokens):
        if tag.startswith('NN'):
            pos = 'n'
        elif tag.startswith('VB'):
            pos = 'v'
        else:
            pos = 'a'
        lemmatized_sentence.append(lemmatizer.lemmatize(word, pos))
    return lemmatized_sentence

def remove_noise(tweet_tokens, stop_words = ()):
    cleaned_tokens = []
    for token, tag in pos_tag(tweet_tokens):
        token = re.sub('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+#]|[!*\(\),]|'\
                       '(?:%[0-9a-fA-F][0-9a-fA-F]))+','', token)
        token = re.sub("(@[A-Za-z0-9_]+)","", token)
        if tag.startswith("NN"):
            pos = 'n'
        elif tag.startswith('VB'):
            pos = 'v'
        else:
            pos = 'a'
        lemmatizer = WordNetLemmatizer()
        token = lemmatizer.lemmatize(token, pos)
        if len(token) > 0 and token not in string.punctuation and token.lower() not in stop_words:
            cleaned_tokens.append(token.lower())
    return cleaned_tokens

positive_cleaned_tokens_list = []
negative_cleaned_tokens_list = []
palantir_cleaned_tokens_list = []

for tokens in positive_tweet_tokens:
    positive_cleaned_tokens_list.append(remove_noise(tokens, stop_words))

for tokens in negative_tweet_tokens:
    negative_cleaned_tokens_list.append(remove_noise(tokens, stop_words))
    
for tokens in palantir_tweet_tokens:
    palantir_cleaned_tokens_list.append(remove_noise(tokens, stop_words))
    
def get_all_words(cleaned_tokens_list):
    for tokens in cleaned_tokens_list:
        for token in tokens:
            yield token

all_pos_words = get_all_words(positive_cleaned_tokens_list)
all_neg_words = get_all_words(negative_cleaned_tokens_list)
all_palantir_words = get_all_words(palantir_cleaned_tokens_list)

#freq_dist_pos = FreqDist(all_pos_words)
#print(freq_dist_pos.most_common(10))

def get_tweets_for_model(cleaned_tokens_list):
    for tweet_tokens in cleaned_tokens_list:
        yield dict([token, True] for token in tweet_tokens)

positive_tokens_for_model = get_tweets_for_model(positive_cleaned_tokens_list)
negative_tokens_for_model = get_tweets_for_model(negative_cleaned_tokens_list)
palantir_tokens_for_model = get_tweets_for_model(palantir_cleaned_tokens_list)

positive_dataset = [(tweet_dict, "Positive")
                     for tweet_dict in positive_tokens_for_model]

negative_dataset = [(tweet_dict, "Negative")
                     for tweet_dict in negative_tokens_for_model]

palantir_positive_dataset = [(tweet_dict, "Positive")
                     for tweet_dict in palantir_tokens_for_model]

palantir_negative_dataset = [(tweet_dict, "Negative")
                     for tweet_dict in palantir_tokens_for_model]

dataset = positive_dataset + negative_dataset
palantir_dataset = palantir_positive_dataset + palantir_negative_dataset

random.shuffle(dataset)
random.shuffle(palantir_dataset)

train_data = dataset[:7000]
test_data = dataset[7000:]

classifier = NaiveBayesClassifier.train(train_data)

print("Accuracy is:", classify.accuracy(classifier, palantir_dataset))

print(classifier.show_most_informative_features(100))

Accuracy is: 0.84
Most Informative Features
                      :( = True           Negati : Positi =   2049.7 : 1.0
                      :) = True           Positi : Negati =   1671.0 : 1.0
                     sad = True           Negati : Positi =     22.7 : 1.0
                follower = True           Positi : Negati =     20.9 : 1.0
                     bam = True           Positi : Negati =     20.6 : 1.0
                     x15 = True           Negati : Positi =     17.4 : 1.0
               community = True           Positi : Negati =     16.6 : 1.0
                followed = True           Negati : Positi =     15.2 : 1.0
                    luck = True           Positi : Negati =     14.5 : 1.0
                     ice = True           Negati : Positi =     14.1 : 1.0
                  arrive = True           Positi : Negati =     13.2 : 1.0
               goodnight = True           Positi : Negati =     12.5 : 1.0
                    miss = True           Negati : Posit